In [ ]:
# Automação Web e Busca de Informações com Python

# Passo 1: Pegar as cotações do dólar, euro e ouro
# ----- abrir o navegador
from selenium import webdriver # dessa forma, eu pego somente uma parte da biblioteca
from selenium.webdriver.common.keys import Keys
import time

navegador = webdriver.Firefox()

# ----- entrar no google
navegador.get("https://google.com.br")

# ----- pesquisar pela cotação
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# ----- pegar a informação da cotação
time.sleep(1)
cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
# teste = navegador.find_element('xpath', '/html/body/div[7]/div/div[10]/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div[3]/div[1]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

# ----- agora vamos fazer para o euro
navegador.get("https://google.com.br")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
time.sleep(1)
cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

# ----- agora para a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)

navegador.quit()

In [ ]:
# ----- Passo 2: Atualizar a base de dados
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")

# atualizar as cotações
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)
display(tabela)

In [ ]:
# ----- Passo 3: Recalcular os preços
# preco de compra = preco original * cotacao
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]
# tabela['Preço de Compra'] = tabela['Preço de Compra'].map("R${:.2f}".format)

# preco de venda = preco de compra * margem
tabela["Preço de Venda"] = tabela['Preço de Compra'] * tabela['Margem']
tabela['Preço de Venda'] = tabela['Preço de Venda'].map("R${:.2f}".format)
display(tabela)

In [ ]:
# Passo 4: Exportar a base de dados
tabela.to_excel("Produtos Novo.xlsx", index=False)